# Openpose 1.7.0 Demo

In [5]:
# 最初に、ランタイムの種類を「GPU」に変更してください。（ランタイム＞ランタイムのタイプを変更）
# GPUが有効になっていることを確認します。

# First, change the runtime type to "GPU".
# Confirm runtime to GPU
#!nvcc --version
!nvidia-smi

Mon Aug 29 22:19:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   46C    P5    14W / 180W |    689MiB /  8192MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# Openpose バージョン指定タグ
# Openpose version tag
ver_openpose = "v1.7.0"
! echo $ver_openpose

v1.7.0


In [7]:
# CMakeが古いとOpenpose（CUDA10)が失敗するので、バージョンを確認します。（BugfixはCMake 3.12.3）
# If CMake is old, Openpose build fails, so download the latest version
# https://developercommunity.visualstudio.com/content/problem/354325/cmake-project-cannot-detect-cuda-10.html
!cmake --version

cmake version 3.22.1

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [8]:
# Openposeのコードをclone
# Clone Openpose
!git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git

Cloning into 'openpose'...
Note: switching to '8ca5c1d95a42340b323e9273654d1db98bec779c'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [17]:
# build用ディレクトリを作成
# Create build directory
! cd openpose && mkdir build && cd build

In [30]:
# https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/installation.md#cmake-command-line-configuration-ubuntu-only
# 上記インストール手順のシナリオ１でインストール実行
# Scenario 1 - Caffe not installed and OpenCV installed using apt-get 
#! cd openpose/build && cmake ..
# COCOモデルのDLオプション付き(「# ! cd」の部分を「! cd」に変更してください。)
# If you want to download the COCO model as well, execute the following command. (Please change "# ! cd" part to "! cd".)
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_COCO_MODEL=ON

# MPIモデルのDLオプション付き（同上）
# If you want to download the MPI model as well, execute the following command.
! cd openpose/build && cmake .. -D DOWNLOAD_BODY_MPI_MODEL=ON

-- Configuring incomplete, errors occurred!
See also "C:/Users/owner/Documents/Python Scripts/04_openpose/openpose/build/CMakeFiles/CMakeOutput.log".


CMake Error at CMakeLists.txt:19 (project):
  Running

   'nmake' '-?'

  failed with:

   謖�螳壹＆繧後◆繝輔ぃ繧､繝ｫ縺瑚ｦ九▽縺九ｊ縺ｾ縺帙ｓ縲�




In [33]:
# Openposeのビルド（15分くらい）
# Openpose Building
! cd openpose/build && make :`nproc`
# outputフォルダ作成
! cd openpose && mkdir output

MAKE Version 5.41  Copyright (c) 1987, 2014 Embarcadero Technologies, Inc.
Error makefile 10: Colon expected
*** 1 errors during make ***


サブディレクトリまたはファイル output は既に存在します。


In [ ]:
# デモ動画を解析
# 出力された解析結果は、Colab画面の左メニューのフォルダアイコンから「openpose/output」以下に配置されます。
# Run and check the sample
# The output analysis result is placed under "openpose/output" from the folder icon on the left menu of the Colab screen.
! cd openpose && ./build/examples/openpose/openpose.bin --video examples/media/video.avi --display 0  --write_video ./output/openpose.avi

In [ ]:
# デモ動画の結果をffmpegで再エンコードして表示します。少し時間がかかります。
# The result of the demo video is re-encoded with ffmpeg and displayed. It will take some time.
! ffmpeg -i "./openpose/output/openpose.avi" -c:v libvpx-vp9 -b:v 1000K -r 30 -threads 8 -speed 1 -tile-columns 6 -frame-parallel 1 -auto-alt-ref 1 -lag-in-frames 25 -c:a libopus -b:a 64k -f webm -y out.webm

from IPython.display import HTML
import base64
import io

video = io.open('./out.webm', 'r+b').read()
encoded = base64.b64encode(video)
data  = '''
<video id="out" width="1280" height="720" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>
'''.format(encoded.decode('ascii'))

HTML(data)

In [ ]:
from google.colab import drive
import os

# Googleドライブマウント
# マウントすると、アクセス許可が求められるので、許可して、キーをコピーして、Colab側の入力欄にCtrl+Vで貼り付けて、Enter。
# mount Google Drive
# When you mount it, you will be asked for permission, so allow it, copy the key, paste(Ctrl+V) it in the input field on the Colab side, and press Enter.
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Google Drive のルートディレクトリ
# Google Drive root directory
root_path = "/gdrive/My Drive/"

# Googleドライブ上で解析したい動画パス（この値を好きな値に変更する事で、ご自分のGoogleドライブ上の動画を解析できます）
# Video path that you want to analyze on Google Drive (You can analyze the video on your own Google Drive by changing this value to a value you like)
file_path = "sample.mp4"

# Googleドライブ上の動画を解析（Googleドライブのルートディレクトリに解析結果の「openpose.avi」が出力されます）
# Analyze video on Google Drive (The analysis result "openpose.avi" is output to the root directory of Google Drive)
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openposed.mp4"

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libth

In [ ]:
# 画像バージョン
image_path = "court.jpg"
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$image_path" --display 0  --write_video "$root_path/openposed.jpg" --

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 8 and format 'image2 / image2 sequence'
OpenPose demo successfully finished. Total time: 3.388164 seconds.


In [ ]:
# Googleドライブ上の解析結果動画をffmpegで再エンコードして表示します。少し時間がかかります。
# The analysis result video on Google Drive is re-encoded with ffmpeg and displayed. It will take some time.
! ffmpeg -i "$root_path/openposed.mp4" -c:v libvpx-vp9 -b:v 1000K -r 30 -threads 8 -speed 1 -tile-columns 6 -frame-parallel 1 -auto-alt-ref 1 -lag-in-frames 25 -c:a libopus -b:a 64k -f webm -y out.webm

from IPython.display import HTML
import base64
import io

video = io.open('./out.webm', 'r+b').read()
encoded = base64.b64encode(video)
data  = '''
<video id="out" width="1280" height="720" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>
'''.format(encoded.decode('ascii'))

HTML(data)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 指定した人数分のみ解析したい場合、「number_people_max」の後ろに人数を指定して下さい
# If you want to analyze only the specified number of people, please specify the number of people after "number_people_max"
# 結果はひとつ上のコードを再実行する事でブラウザ上で確認できます。
# The result can be confirmed on the browser by re-executing the code one level above.
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi" --number_people_max 1

In [ ]:
# 手の解析も一緒に行いたい場合、「--hand」を追加してください
# If you also want to analyze the hands together, add "--hand"
# 結果は２つ上のコードを再実行する事でブラウザ上で確認できます。
# The result can be confirmed on the browser by re-executing the code two level above.
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi" --hand

In [ ]:
# 顔の解析も一緒に行いたい場合、「--face」を追加してください
# If you also want to analyze the hands together, add "--hand"
# 結果は３つ上のコードを再実行する事でブラウザ上で確認できます。
# The result can be confirmed on the browser by re-executing the code three level above.
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi" --face